In [24]:
# ensure that any code changes are immediately reflected
%reload_ext autoreload
%autoreload 2

## Initialize Three Users

In [4]:
from mostlyai import MostlyAI
# Test User 1
mostly_t1 = MostlyAI(api_key='mostly-020bd715ef9abca16e07bc800fc31126276f1508baab20036ee08315ceef8527')
# Test User 2
mostly_t2 = MostlyAI(api_key='mostly-fc5d7602dba784ce687d775b72c12d2e93407fba9731144145b85e0548ac1e97')
# Test User 3
mostly_t3 = MostlyAI(api_key='mostly-0dd57f8b8cdb7485310aadead0439c48beea8956273614102e8bf7950bc3f02e')

## Basic Usage

### User 1 trains a Generator

In [5]:
g = mostly_t1.train('https://raw.githubusercontent.com/mostly-ai/public-demo-data/dev/titanic/titanic.csv')

Created generator ]8;id=281354;https://llb2.dev.mostlylab.com/d/generators/d28b2d4f-6af4-4e21-9cca-a7ba34d25535\d28b2d4f-6af4-4e21-9cca-a7ba34d25535]8;;\

Started generator training

Output()

🎉 Your generator is ready! Use it to create synthetic data. Share it so others can do the same.

In [25]:
g

{
    'id': UUID('d28b2d4f-6af4-4e21-9cca-a7ba34d25535'),
    'name': 'titanic',
    'description': None,
    'training_status': <ProgressStatus.done: 'DONE'>,
    'metadata': {
        'created_at': datetime.datetime(2024, 2, 27, 9, 29, 30, 827227, tzinfo=TzInfo(UTC)),
        'owner_id': '94b142d1-f543-40a2-98ba-3192a738c4ee',
        'owner_name': 'Test 1',
        'current_user_permission_level': <PermissionLevel.admin: 'ADMIN'>
    },
    'accuracy': 0.922,
    'usage': None,
    'tables': [
        {
            'id': UUID('3e3cc82b-7d69-4c4e-8538-6889df7a143b'),
            'source_connector': {
                'id': UUID('b3796a35-1288-4ea6-bba5-016ad1fb70e5'),
                'name': None,
                'uri': '/connectors/b3796a35-1288-4ea6-bba5-016ad1fb70e5'
            },
            'location': None,
            'name': 'titanic',
            'primary_key': None,
            'columns': [
                {
                    'id': UUID('94f431c0-22bd-4ebf-be01-fe3a9db49659'),
                    'name': 'survived',
                    'included': True,
                    'model_encoding_type': <ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>
                },
                {
                    'id': UUID('24351843-5edf-4575-8056-bf5d0bf2d5e7'),
                    'name': 'pclass',
                    'included': True,
                    'model_encoding_type': <ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>
                },
                {
                    'id': UUID('b13328f4-d3ab-441c-8f91-9759c7e15ae7'),
                    'name': 'sex',
                    'included': True,
                    'model_encoding_type': <ModelEncodingType.categorical: 'CATEGORICAL'>
                },
                {
                    'id': UUID('b5ed9195-3ea5-4480-99aa-dcfc9fb182eb'),
                    'name': 'age',
                    'included': True,
                    'model_encoding_type': <ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>
                },
                {
                    'id': UUID('a5ce1710-4fde-439d-b3eb-9995a7e15cd3'),
                    'name': 'sibsp',
                    'included': True,
                    'model_encoding_type': <ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>
                },
                {
                    'id': UUID('a9dedf8e-4fde-4fdb-8a6c-45a7f875f256'),
                    'name': 'parch',
                    'included': True,
                    'model_encoding_type': <ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>
                },
                {
                    'id': UUID('57fc20a4-c900-4ccd-98b7-904e0206a374'),
                    'name': 'fare',
                    'included': True,
                    'model_encoding_type': <ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>
                },
                {
                    'id': UUID('20000239-6d4c-4945-bd3c-95a2878502bc'),
                    'name': 'embarked',
                    'included': True,
                    'model_encoding_type': <ModelEncodingType.categorical: 'CATEGORICAL'>
                }
            ],
            'foreign_keys': None,
            'model': {
                'id': 'a5793199-b459-435a-8f81-72724faa412b',
                'configuration': {
                    'sample_fraction': 1.0,
                    'max_sample_size': None,
                    'batch_size': None,
                    'model_size': <ModelSize.m: 'M'>,
                    'max_training_time': 10,
                    'max_epochs': 100,
                    'max_sequence_window': 100,
                    'enable_flexible_generation': True,
                    'value_protection': True,
                    'rare_category_replacement_method': <RareCategoryReplacementMethod.constant: 'CONSTANT'>
                },
                'metrics': {
                    'accuracy': {
                        'overall': 0.922,
                        'univariate': 0.949,
               

Generator(id=UUID('d28b2d4f-6af4-4e21-9cca-a7ba34d25535'), name='titanic', description=None, training_status=<ProgressStatus.done: 'DONE'>, metadata=Metadata(created_at=datetime.datetime(2024, 2, 27, 9, 29, 30, 827227, tzinfo=TzInfo(UTC)), owner_id='94b142d1-f543-40a2-98ba-3192a738c4ee', owner_name='Test 1', current_user_permission_level=<PermissionLevel.admin: 'ADMIN'>), accuracy=0.922, usage=None, tables=[SourceTable(id=UUID('3e3cc82b-7d69-4c4e-8538-6889df7a143b'), source_connector=BaseResource(id=UUID('b3796a35-1288-4ea6-bba5-016ad1fb70e5'), name=None, uri='/connectors/b3796a35-1288-4ea6-bba5-016ad1fb70e5'), location=None, name='titanic', primary_key=None, columns=[SourceColumn(id=UUID('94f431c0-22bd-4ebf-be01-fe3a9db49659'), name='survived', included=True, model_encoding_type=<ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>), SourceColumn(id=UUID('24351843-5edf-4575-8056-bf5d0bf2d5e7'), name='pclass', included=True, model_encoding_type=<ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>), SourceColumn(id=UUID('b13328f4-d3ab-441c-8f91-9759c7e15ae7'), name='sex', included=True, model_encoding_type=<ModelEncodingType.categorical: 'CATEGORICAL'>), SourceColumn(id=UUID('b5ed9195-3ea5-4480-99aa-dcfc9fb182eb'), name='age', included=True, model_encoding_type=<ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>), SourceColumn(id=UUID('a5ce1710-4fde-439d-b3eb-9995a7e15cd3'), name='sibsp', included=True, model_encoding_type=<ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>), SourceColumn(id=UUID('a9dedf8e-4fde-4fdb-8a6c-45a7f875f256'), name='parch', included=True, model_encoding_type=<ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>), SourceColumn(id=UUID('57fc20a4-c900-4ccd-98b7-904e0206a374'), name='fare', included=True, model_encoding_type=<ModelEncodingType.numeric_auto: 'NUMERIC_AUTO'>), SourceColumn(id=UUID('20000239-6d4c-4945-bd3c-95a2878502bc'), name='embarked', included=True, model_encoding_type=<ModelEncodingType.categorical: 'CATEGORICAL'>)], foreign_keys=None, model=Model(id='a5793199-b459-435a-8f81-72724faa412b', configuration=ModelConfiguration(sample_fraction=1.0, max_sample_size=None, batch_size=None, model_size=<ModelSize.m: 'M'>, max_training_time=10, max_epochs=100, max_sequence_window=100, enable_flexible_generation=True, value_protection=True, rare_category_replacement_method=<RareCategoryReplacementMethod.constant: 'CONSTANT'>), metrics=ModelMetrics(accuracy=Accuracy(overall=0.922, univariate=0.949, bivariate=0.894, coherence=None, overall_max=0.959), distances=Distances(dcr_original=0.679, dcr_synthetic=1.311))), text_model=None, total_rows=1309)], synthetic_datasets=[], training=<mostlyai.model.Generator.Training object at 0x120f6d0c0>)

### User 1 shares the Generator with User 2 and User 3

In [ ]:
mostly_t1.share(g, 'test2@mostly.ai')
mostly_t1.share(g, 'test3@mostly.ai')

### User 2 generates a Synthetic Dataset

In [ ]:
sd = mostly_t2.generate(g.id, size=1000)

### User 2 consumes the Synthetic Dataset

In [ ]:
sd.data()

### User 3 generates a Seeded Synthetic Dataset

In [ ]:
# prepare a seed DataFrame
import pandas as pd
import numpy as np
seed_df = pd.DataFrame({'survived': [1] * 10_000, 'age': np.random.randint(5, 18, 10_000)})
seed_df

In [ ]:
sd3 = mostly_t3.generate(g.id, seed=seed_df)

In [ ]:
sd3.data()

## Multi-Table

### User 1 trains a 3-table setup from pandas

In [ ]:
import pandas as pd
df_players = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/baseball/players.csv.gz')
df_fielding = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/baseball/fielding.csv.gz')
df_batting = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/baseball/batting.csv.gz')
df_players

In [ ]:
g = mostly_t1.train(config=
    {
        "name": "baseball",
        "tables": [
            {
                "name": "players", 
                "data": df_players, 
                "primaryKey": "id",
                "modelConfiguration": {"maxTrainingTime": 1}
            },
            {
                "name": "fielding",
                "data": df_fielding, 
                "foreignKeys": [{"column": "players_id", "referencedTable": "players", "isContext": True}],
                "modelConfiguration": {"maxTrainingTime": 1}
            },
            {
                "name": "batting",
                "data": df_batting, 
                "foreignKeys": [{"column": "players_id", "referencedTable": "players", "isContext": True}],
                "modelConfiguration": {"maxTrainingTime": 1}
            }
        ]
    }
)

## Database Connectivity

### User 1 creates a Connector to a Postgres Database

In [ ]:
import os
c = mostly_t1.connect({
    "accessType": "SOURCE",
    "type": "POSTGRES",
    "config": {
        "host": "postgres-test.cppq7davohkj.eu-central-1.rds.amazonaws.com",
        "username": "postgres",
        "database": "berka_original",
    },
    "secrets": {
        "password": pwd
    }
})

### User 1 shares Connector with User 2

In [ ]:
mostly_t1.share(c.id, 'test2@mostly.ai')

### User 2 uses Connector to train a Generator

In [ ]:
c2 = mostly_t2.connectors.get(c.id)
c2

In [ ]:
c2.locations()

In [ ]:
c2.locations("berka")

In [ ]:
g = mostly_t2.train(config=
    {
        "name": "BERKA",
        "tables": [
            {
                "name": "accounts", 
                "sourceConnectorId": str(c2.id),
                "location": "berka.account",
                "primaryKey": "account_id"
            }
        ]
    }
)

In [ ]:
sd = mostly_t2.generate(g)

In [ ]:
sd.data()